In [54]:
import ciropt as co
import cvxpy as cp

In [55]:
L_smooth = 1.
mu = 0.0001

Capacitance = 2.
Inductance = 2.
R = 1

# solver = "ca"
solver = "ca_canonical"
# solver = "ca_canonical_X"
# solver = "gp_canonical_X"

time_limit = 200

# Ciropt problem

In [56]:
from ciropt.circuit_algorithms import define_function
from ciropt.circuit_opt import CircuitOpt
from ciropt.function import SmoothStronglyConvexFunction, StronglyConvexFunction, \
                            SmoothConvexFunction, ConvexFunction

problem = CircuitOpt()
func = define_function(problem, mu, L_smooth)

h, alpha, beta, b = problem.h, problem.alpha, problem.beta, problem.b
x_star, y_star, f_star = func.stationary_point()


# stepsize : CR

# x_1 = problem.set_initial_point()
# v_C_1 = x_1 + R * y_1
# y_1, f_1 = func.oracle(x_1)

v_C_1 = problem.set_initial_point()
x_1, y_1, f_1 = func.proximal_step(v_C_1, R)
i_C_1 = 1/R * (x_1 - v_C_1)

v_C_1p5 = v_C_1 + alpha * R * h  * i_C_1
x_1p5, y_1p5, f_1p5 = func.proximal_step(v_C_1p5, R)
i_C_1p5 = 1/R * (x_1p5 - v_C_1p5)

v_C_2 = v_C_1p5 + beta * R * h  * i_C_1 + (1 - beta) * R * h * i_C_1p5


E_1 = (Capacitance/2) * (v_C_1 - x_star)**2
E_2 = (Capacitance/2) * (v_C_2 - x_star)**2
Delta_1 = b * (f_1 - f_star)
problem.set_energy(E_1 = E_1, E_2 = E_2)
problem.set_delta( Delta_1 )
problem.obj = b


In [57]:
# problem = co.gradient_flow_circuit(mu, L_smooth, Capacitance)

res, model, sp_exp = problem.solve(solver=solver, verbose=False, time_limit=100)[:3]
print(res)

dim_G=4, dim_F=3
{'b': 6.000200008379675, 'h': 2.5975637852687723, 'd': 0.0, 'alpha': 0.15488002295233472, 'beta': 0.28695288407585734}


In [58]:
res

{'b': 6.000200008379675,
 'h': 2.5975637852687723,
 'd': 0.0,
 'alpha': 0.15488002295233472,
 'beta': 0.28695288407585734}

# PEP verification

In [65]:
import PEPit
from PEPit.functions import SmoothStronglyConvexFunction, SmoothConvexFunction
from PEPit.primitive_steps import proximal_step

b = res["b"] - 3
h = res["h"]
alpha = res["alpha"]
beta = res["beta"]

In [66]:
problem = PEPit.PEP()
func_P = problem.declare_function(
                    SmoothStronglyConvexFunction,
                    mu=mu,   # Strong convexity param.
                    L=L_smooth) 


x_P_star = func_P.stationary_point()
g_star, f_star = func_P.oracle(x_P_star)


# x_P_1 = problem.set_initial_point()
# g_1, f_1 = func.oracle(x_P_1)
# v_P_C_1 = x_P_1 + R * g_1

v_P_C_1 = problem.set_initial_point()
x_P_1, y_P_1, f_P_1 = proximal_step(v_P_C_1, func_P, R)
# x_P_1, y_P_1, f_P_1 = func.proximal_step(v_P_C_1, R)
i_P_C_1 = 1/R * (x_P_1 - v_P_C_1)

v_P_C_1 = x_P_1 + R * y_P_1
i_P_C_1 = 1/R * (x_P_1 - v_P_C_1)

v_P_C_1p5 = v_P_C_1 + alpha * R * h  * i_P_C_1
x_P_1p5, y_P_1p5, f_P_1p5 = proximal_step(v_P_C_1p5, func_P, R)
# x_1p5, y_1p5, f_1p5 = func.proximal_step(v_C_1p5, R)
i_P_C_1p5 = 1/R * (x_P_1p5 - v_P_C_1p5)

v_P_C_2 = v_P_C_1p5 + beta * R * h  * i_P_C_1 + (1 - beta) * R * h * i_P_C_1p5


# Set the performance metric to the function values accuracy
E_1 = (Capacitance/2) * (v_P_C_1 - x_P_star)**2
E_2 = (Capacitance/2) * (v_P_C_2 - x_P_star)**2
Delta_1 = b * (f_P_1 - f_star)
problem.set_performance_metric(E_2 - (E_1 - Delta_1))

# Solve the PEP with verbose = 1
verbose = 0
diff = problem.solve(verbose=verbose, solver=cp.MOSEK)
print(f"{diff=}")

diff=-1.5267189849540714e-12
